In [17]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import glob
import os
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statistics
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import plotly.express as px
from plotly.subplots import make_subplots

In [18]:
import plotly.io as pio
pio.renderers.default = "vscode"
pio.renderers.default = "notebook_connected"

In [19]:
path = "../data/"
experments = ["exp1","exp2"]
system = ["flink","spark"]
#os.listdir("{}/{}/{}/".format(path, experments[1], system[0]))

In [20]:
exp1_flink = pd.concat(map(pd.read_csv, glob.glob(os.path.join(path, experments[0], system[0],"*.csv"))), ignore_index=True)
exp1_spark = pd.concat(map(pd.read_csv, glob.glob(os.path.join(path, experments[0], system[1],"*.csv"))), ignore_index=True)  
exp2_flink = pd.concat(map(pd.read_csv, glob.glob(os.path.join(path, experments[1], system[0],"*.csv"))), ignore_index=True)
exp2_spark = pd.concat(map(pd.read_csv, glob.glob(os.path.join(path, experments[1], system[1],"*.csv"))), ignore_index=True)  
    


In [21]:
exp1_flink.rename(columns={"Unnamed: 0":"testIndex","0":"countOfEvent","1":"latencyTime","2":"NumberOfNodes","3":"TestNumber","4":"System"},inplace=True)
exp1_spark.rename(columns={"Unnamed: 0":"testIndex","0":"countOfEvent","1":"latencyTime","2":"NumberOfNodes","3":"TestNumber","4":"System"},inplace=True)
exp2_flink.rename(columns={"Unnamed: 0":"testIndex","0":"countOfEvent","1":"latencyTime","2":"NumberOfNodes","3":"TestNumber","4":"System"},inplace=True)
exp2_spark.rename(columns={"Unnamed: 0":"testIndex","0":"countOfEvent","1":"latencyTime","2":"NumberOfNodes","3":"TestNumber","4":"System"},inplace=True)


In [22]:
exp1 = pd.concat([exp1_flink,exp1_spark])

In [23]:
exp1_flink.groupby('NumberOfNodes')['latencyTime'].describe()

,count,mean,std,min,25%,50%,75%,max
NumberOfNodes,,,,,,,,
2,34100.0,10329.097038,1894.928040,487.0,10355.0,10637.0,10870.0,25271.0
4,51486.0,10118.271064,1675.411323,253.0,10255.0,10511.0,10769.0,11416.0
8,40400.0,10204.211287,1653.691489,427.0,10298.0,10644.0,10843.0,11369.0


In [24]:
exp1_spark.groupby('NumberOfNodes')['latencyTime'].describe()

,count,mean,std,min,25%,50%,75%,max
NumberOfNodes,,,,,,,,
2,33531.0,29243.770749,17539.099506,11156.0,13038.0,22583.0,44988.00,68821.0
4,62104.0,14211.018694,4062.633293,10517.0,11362.0,12115.0,15984.25,28197.0
8,68875.0,10665.424261,1355.789015,1078.0,10644.0,10841.0,11100.00,13712.0


Perform 2 way avova to see if we have a stat diff in mean latencyTime units milliseconds. 
Assumptions: Normality, common vaariance, drawn independently, samples without are random and independed, factor effect are additive
Assumptions should be met will these varrances. 

In [25]:
print("Spark Varance")
print(exp1_spark.groupby('NumberOfNodes')['latencyTime'].var())
print("flink varance")
print(exp1_flink.groupby('NumberOfNodes')['latencyTime'].var())

Spark Varance
NumberOfNodes
2    3.076200e+08
4    1.650499e+07
8    1.838164e+06
Name: latencyTime, dtype: float64
flink varance
NumberOfNodes
2    3.590752e+06
4    2.807003e+06
8    2.734696e+06
Name: latencyTime, dtype: float64


In [26]:
model_exp1 = ols('latencyTime ~ C(NumberOfNodes) + C(System) + C(NumberOfNodes):C(System)', data = exp1).fit()
sm.stats.anova_lm(model_exp1, typ=2)

,sum_sq,df,F,PR(>F)
C(NumberOfNodes),4.338245e+12,2.0,53202.672626,0.0
C(System),2.830014e+12,1.0,69412.538631,0.0
C(NumberOfNodes):C(System),3.695492e+12,2.0,45320.176665,0.0
Residual,1.184355e+13,290490.0,NaN,NaN


We can reject null between these group there is a statistaically signification difference between these groups. in order ot find out which groups we will perfrom a tukey's Test   

In [27]:
exp1['groups'] = exp1.apply(lambda row: str(row.NumberOfNodes) + row.System, axis=1)

In [28]:
exp1.head()

,testIndex,countOfEvent,latencyTime,NumberOfNodes,TestNumber,System,groups
0,0,183.0,5773.0,2,26,flink,2flink
1,1,339.0,10756.0,2,26,flink,2flink
2,2,349.0,10722.0,2,26,flink,2flink
3,3,332.0,10692.0,2,26,flink,2flink
4,4,336.0,10665.0,2,26,flink,2flink


In [29]:
tukey_exp1 = pairwise_tukeyhsd(endog = exp1['latencyTime'], groups=exp1['groups'], alpha=0.05)
print(tukey_exp1)

   Multiple Comparison of Means - Tukey HSD,FWER=0.05   
group1 group2   meandiff     lower       upper    reject
--------------------------------------------------------
2flink 2spark  18914.6737  18774.7327  19054.6147  True 
2flink 4flink   -210.826   -337.8694    -83.7825   True 
2flink 4spark  3881.9217   3759.2816   4004.5617   True 
2flink 8flink  -124.8858    -258.694     8.9225   False 
2flink 8spark   336.3272    215.8429    456.8115   True 
2spark 4flink -19125.4997 -19253.1899 -18997.8094  True 
2spark 4spark -15032.7521  -15156.062 -14909.4421  True 
2spark 8flink -19039.5595  -19173.982  -18905.137  True 
2spark 8spark -18578.3465 -18699.5126 -18457.1803  True 
4flink 4spark  4092.7476   3984.2954   4201.1998   True 
4flink 8flink   85.9402     -34.9976    206.878   False 
4flink 8spark   547.1532    441.1448    653.1616   True 
4spark 8flink  -4006.8074  -4123.1108  -3890.504   True 
4spark 8spark  -3545.5944  -3646.2836  -3444.9052  True 
8flink 8spark   461.213     347

help with anlyzing https://www.statology.org/tukey-test-python/
Pull attention to its intersting in our data we dont see increasing the about of flink nodes. There is a different going from 2 flink nodes to 4 flink nodes but going form 2 to 8 or 4 to 8 we cannot reject the null. 


In [30]:
exp1_spark[exp1_spark["NumberOfNodes"] == 2].head()

,testIndex,countOfEvent,latencyTime,NumberOfNodes,TestNumber,System
2455,0,288.0,58944.0,2,35,spark
2456,1,319.0,47793.0,2,35,spark
2457,2,332.0,31165.0,2,35,spark
2458,3,319.0,22235.0,2,35,spark
2459,4,334.0,16008.0,2,35,spark


In [37]:

exp1_spark['latencyTime(s)'] = exp1_spark.apply(lambda row: row.latencyTime / 1000, axis=1)

exp1_flink['latencyTime(s)'] = exp1_flink.apply(lambda row: row.latencyTime / 1000, axis=1)

exp1['latencyTime(s)'] = exp1.apply(lambda row: row.latencyTime / 1000, axis=1)


In [32]:
spark2 = px.histogram(exp1_spark[exp1_spark["NumberOfNodes"] == 2],  x = "countOfEvent", y = "latencyTime(s)", histfunc='avg', title="Exp 1 Spark 2 Nodes")
spark2.update_layout(xaxis_title="Times(s)", yaxis_title="Latency(s)")

spark4 = px.histogram(exp1_spark[exp1_spark["NumberOfNodes"] == 4],  x = "countOfEvent", y = "latencyTime(s)", histfunc='avg',title="Exp 1 Spark 4 Nodes")
spark4.update_layout(xaxis_title="Times(s)", yaxis_title="Latency(s)")

spark8 = px.histogram(exp1_spark[exp1_spark["NumberOfNodes"] == 8],  x = "countOfEvent", y = "latencyTime(s)", histfunc='avg', title="Exp 1 Spark 8 Nodes")
spark8.update_layout(xaxis_title="Times(s)", yaxis_title="Latency(s)")

if not os.path.exists("graphs"):
    os.mkdir("graphs")

spark2.write_image("graphs/exp1_spark_n2.jpeg")
spark4.write_image("graphs/exp1_spark_n4.jpeg")
spark8.write_image("graphs/exp1_spark_n8.jpeg")



In [36]:
flink2 = px.histogram(exp1_flink[exp1_flink["NumberOfNodes"] == 2],  x = "countOfEvent", y = "latencyTime(s)", histfunc='avg', title="Exp 1 Flink 2 Nodes")
flink2.update_layout(xaxis_title="Times(s)", yaxis_title="Latency(s)")

flink4 = px.histogram(exp1_flink[exp1_flink["NumberOfNodes"] == 4],  x = "countOfEvent", y = "latencyTime(s)", histfunc='avg',title="Exp 1 Flink 4 Nodes")
flink4.update_layout(xaxis_title="Times(s)", yaxis_title="Latency(s)")

flink8 = px.histogram(exp1_flink[exp1_flink["NumberOfNodes"] == 8],  x = "countOfEvent", y = "latencyTime(s)", histfunc='avg', title="Exp 1 Flink 8 Nodes")
flink8.update_layout(xaxis_title="Times(s)", yaxis_title="Latency(s)")

if not os.path.exists("graphs"):
    os.mkdir("graphs")

flink2.write_image("graphs/exp1_flink_n2.jpeg")
flink4.write_image("graphs/exp1_flink_n4.jpeg")
flink8.write_image("graphs/exp1_flink_n8.jpeg")


In [ ]:
exp1_box = px.box(exp1, x = "NumberOfNodes", y="latencyTime(s)", color="Sysem")

In [34]:
exp2_flink.groupby('NumberOfNodes')['latencyTime'].describe()

,count,mean,std,min,25%,50%,75%,max
NumberOfNodes,,,,,,,,
2,41012.0,9866.224568,2078.708760,653.0,10254.0,10602.0,10815.0,11740.0
4,71564.0,9918.743265,1935.103993,687.0,10210.0,10524.0,10826.0,11507.0
8,27600.0,9817.799457,2057.931267,439.0,10218.0,10497.0,10750.0,11709.0


In [35]:
exp2_spark.groupby('NumberOfNodes')['latencyTime'].describe()

,count,mean,std,min,25%,50%,75%,max
NumberOfNodes,,,,,,,,
2,20203.0,29139.413602,19332.481246,8420.0,12502.0,18225.0,46179.0,66463.0
4,24987.0,14350.764317,5482.340717,3240.0,11174.0,12028.0,17297.5,31786.0
8,31800.0,10519.105000,1796.317631,2737.0,10601.0,10874.0,11204.0,13595.0
